In [1]:
import hexathello.aiPlayers as aiPlayers
import hexathello.autoPlayer as autoPlayer
import hexathello.engine as engine
import hexathello.history as history
import hexathello.jable as jable
import hexathello.printing as printing

import numpy as np

from os import path

# -- Settings
game_size: int = 5
player_count: int = 2

In [2]:
# We want to train a KerasHexAgent. To do this, we use data we created in `quickstart_recording_data.ipynb`

history_dir: str = path.join(
    'data',
    'history',
    'examples'
)
assert path.isdir(
    history_dir
)

baseline_data_path: str = path.join(
    history_dir,
    'greendom_size-{}_players-{}.json'.format(
        game_size, player_count
    )
)
   

In [3]:
# Read the data from disk to learn from
history_fromDisk: jable.JyFrame = jable.read_file(
    baseline_data_path
)

# Decode the state, option, and play vectors from integers to numpy arrays
history_decoded: jable.JyFrame = history.history_fromInt(
    history_fromDisk
)

assert len( history_decoded ) >= 20000
    
# Make it PoV 0 to appropriately learn
povHistory: jable.JyFrame = history.povHistory_from_literalHistory(
    history_decoded
)

del history_decoded
del history_fromDisk

In [4]:
# We want to train a Keras Neural Network on the data we have.
# The input size is the length of a state vector
# The output size is the length of the play vector
# Take both from the first row
input_size: int = len( povHistory[0,'board_state'] )
output_size: int = len( povHistory[0, 'player_action'] )

# The `KerasHexAgent` subclass of `HexAgent` has a `brain` property; this is the neural network
# We could in fact us any objects conforming to the `PredictionModel` protocol, which has methods:
#   - fit()
#   - predict()
#   - call()
#
# We're going to train it on the Greendom data
# Match the input to a board state vector
import tensorflow as tf

ai_keras_id: str = 'kha_alpha_size-{}_players-{}_0'.format(
    game_size, player_count
)

ai_keras_path: str = path.join(
    'data',
    'ai',
    'examples',
    '{}.keras'.format( ai_keras_id )
)

brain_model: tf.keras.Model
if path.isfile( ai_keras_path ):
    brain_model = tf.keras.models.load_model( ai_keras_path )
#
else:
    brain_input = tf.keras.layers.Input(
        shape = (input_size,),
        name = 'keras_tensor'
    )

    # Get creative with architecture on the inside
    brain_next = tf.keras.layers.Dense(
        input_size*2,
        activation = 'relu'
    )( brain_input )

    brain_next = tf.keras.layers.Dense(
        input_size*2,
        activation = 'relu'
    )( brain_next )

    # Make the output size equal to the move vector size
    brain_output = tf.keras.layers.Dense(
        output_size,
        activation = 'sigmoid'
    )( brain_next )

    brain_model: tf.keras.Model = tf.keras.Model(
        brain_input,
        brain_output
    )

    # Choose your learning rate and optimizer. Adam is probably good for the latter.
    # You most likely want Binary Cross Entropy. Learning rate 0.0001 to 0.01 is likely fine
    brain_model.compile(
        optimizer = tf.keras.optimizers.Adam(
            learning_rate = 0.005
        ),
        loss = tf.keras.losses.CategoricalCrossentropy()
    )
#/if path.isfile( ai_keras_path )

# Init the AI Agent
ai_keras: aiPlayers.KerasHexAgent = aiPlayers.KerasHexAgent(
    size = game_size,
    player_count = player_count,
    brain = brain_model,
    player_id = None,
    ai_id = ai_keras_id
)

# Set the checkpoint to save
ai_keras_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=ai_keras_path,
    monitor='loss',
    mode='min',
    save_best_only=True
)

ai_keras.train(
    game_history = povHistory,
    epochs = 10,
    callbacks = [ ai_keras_checkpoint_callback ]
)



2025-03-31 17:45:33.537478: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/10
626/626 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.3537
Epoch 2/10
626/626 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.3337
Epoch 3/10
626/626 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.2862
Epoch 4/10
626/626 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.2682
Epoch 5/10
626/626 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.2488
Epoch 6/10
626/626 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.2466
Epoch 7/10
626/626 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.2757
Epoch 8/10
626/626 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.2483
Epoch 9/10
626/626 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.2166
Epoch 10/10
626/626 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.2470


In [5]:
# Save the network
ai_keras.brain.summary()


# Get creative with the brain you use to train a KerasHexAgent, and try writing a subclass changing:
#   .prep_training_history(...)
#   .chooseMove(...)

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ keras_tensor (InputLayer)       │ (None, 122)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 244)            │        30,012 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 244)            │        59,780 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 61)             │        14,945 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 314,213 (1.20 MB)

 Trainable params: 104,737 (409.13 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 209,476 (818.27 KB)